<a href="https://colab.research.google.com/github/Pert814/Python_Basic/blob/main/1_14%E7%88%AC%E8%9F%B2%E8%AA%B2%E7%A8%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HTTP代碼

```python
2開頭: 成功

3開頭: 轉址
對面的網址已經搬家了, 雖然妳還是輸入舊網址, 但她會自動幫你替換成新網址

4開頭: 錯誤

404: Not Found
!!! 最常見: 網址打錯
403: Forbidden
!!! 原因1: ip ban
!!! 通常是數次的惡意行為(可能是短時間送太多...)
!!! 解法: 等待/換ip
!!! 原因2: 妳學的不像瀏覽器
!!! 解法: 學的像一點(headers完善)

```

# 新增區段

Header

```python
妳在送出網址的時候, 其實不只送出網址
妳送出的是 網址+額外信息(request headers)

妳在得到回應的時候, 其實不是只有得到回應
回應+額外信息(response headers)
```

處理檔案

```python
1. 純文字(TXT)
open(...., "r" or "w", encoding="utf-8")

2. 非純文字(圖片/pdf/doc....)
open(...., "rb" or "wb")

```

In [ ]:
import urllib.request as req
import bs4 as bs

url = "https://www.ptt.cc/bbs/Beauty/M.1736755829.A.02A.html"
r = req.Request(url)
r.add_header("user-agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36")

response = req.urlopen(r)
soup = bs.BeautifulSoup(response)

links = soup.find_all("a")
allow_subname = ["jpg", "jpeg", "gif"]

for l in links:
  href = l["href"]
  subname = href.split(".")[-1]
  if subname.lower() in allow_subname:
    r = req.Request(href)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0")
    img = req.urlopen(r)
    # print(img.read())
    # print("-" * 30)
    fname = href.split("/")[-1]
    f =  open(fname)

FileNotFoundError: [Errno 2] No such file or directory: '8faNwgb.jpeg'

In [ ]:
import urllib.request as req
import bs4 as bs

url = "https://www.ptt.cc/bbs/Beauty/M.1736755829.A.02A.html"
r = req.Request(url)
r.add_header("user-agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36")
response = req.urlopen(r)
soup = bs.BeautifulSoup(response)
# print(soup)

# 抓取<span class="article-meta-value">Gentlemon (肥宅紳士)</span>
author_html = soup.find_all("span", class_="article-meta-value")
# print(author_html[0].text)
title_split = author_html[2].text.split("]")
title_class = title_split[0].replace("[", "")
title_board = title_split[1].strip(" ")

# # 抓取<title>[正妹] Cosplay 1601 日本 約兒 - 看板 Beauty - 批踢踢實業坊</title>
# title_html = soup.find("title")
# title = title_html.text.strip(" ")

# # 抓取<span class="article-meta-tag">時間</span><span class="article-meta-value">Mon Jan 13 16:10:23 2025</span>
# time_html = soup.find("span", class_="article-meta-value")
# time = time_html.text.strip(" ")

# #抓取<span class="article-meta-value">Beauty</span>
# board_html = soup.find_all("span", class_="article-meta-value")
# print(board_html)
# # board = board_html.text.strip(" ")

# 抓取所有<span class="f3 push-content">: 大長腿</span>
push_html = soup.find_all("span", class_="f3 push-content")
push_content = []
for i in push_html:
  push_content.append(i.text)

print("作者: ", author_html[0].text)
print("看板: ", author_html[1].text)
print("分類: ", title_class)
print("標題: ", title_board)
print("時間: ", author_html[3].text)
print("推噓文", push_content)


作者:  Gentlemon (肥宅紳士)
看板:  Beauty
分類:  正妹
標題:  Cosplay 1601 日本 約兒
時間:  Mon Jan 13 16:10:23 2025
推噓文 [': 大長腿', ': 好！！！', ': 長', ': 讚', ': 可']


In [ ]:
# 練習: 拿其他的
import urllib.request as req
import bs4 as bs

url = "https://www.ptt.cc/bbs/Beauty/M.1736733511.A.738.html"
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0")

response = req.urlopen(r)
html = bs.BeautifulSoup(response)

meta_list = html.find_all("span", {"class":"article-meta-value"})

ptt_id = meta_list[0].text
if "(" in ptt_id:
    ptt_id_split = ptt_id.split("(")
    ptt_id = ptt_id_split[0].strip()
    nick_name = ptt_id_split[-1].replace(")", "")
else:
    nick_name = ""
board = meta_list[1].text
title = meta_list[2].text
if title[0] == "[":
    title_split = title.split("]")
    category = title_split[0].replace("[", "")
    title = title_split[-1].strip()
else:
    category = ""
date = meta_list[3].text

# 存檔的時候肯定是來個JSON
data = {
    "id":ptt_id,
    "nickname":nick_name,
    "board":board,
    "category":category,
    "title":title,
    "data":date,
    "push":[]
}

print("作者:", ptt_id)
print("當時暱稱:", nick_name)
print("看板:", board)
print("分類:", category)
print("標題:", title)
print("時間:", date)


push_list = html.find_all("div", {"class":"push"})
for push in push_list:
    push_meta = push.find_all("span")
    push_type = push_meta[0].text.strip()
    push_ptt_id = push_meta[1].text.strip()
    push_content = push_meta[2].text.replace(": ", "").strip()
    push_ip_date = push_meta[3].text.strip()
    print(push_type, push_ptt_id, push_content, push_ip_date)
    push_data = {
        "type":push_type,
        "id":push_ptt_id,
        "content":push_content,
        "ipdate":push_ip_date
    }
    data["push"].append(push_data)

作者: JANUARZ
當時暱稱: 社會職人
看板: Beauty
分類: 正妹
標題: 有些角度像楊謹華
時間: Mon Jan 13 09:58:29 2025
推 ninaman 正 101.12.146.84 01/13 10:05
推 deltarobot . 49.217.122.9 01/13 10:07
推 elfindor 優 223.137.175.186 01/13 10:16
推 wglhe 優版派克 42.77.77.102 01/13 10:17
推 Uncontinue 正正 122.118.35.37 01/13 10:38
推 Williamtsou 門 27.53.230.154 01/13 10:52
推 aass5566 就是本人 42.77.55.73 01/13 11:10
→ bingreen 正 111.184.234.172 01/13 11:17
推 openbook13 優質皮朋 49.215.58.105 01/13 11:29
推 okbon 不錯喔 118.231.152.241 01/13 13:03
噓 TopGun2 明明就是 張鳳書 114.33.106.198 01/13 13:12
→ wl00669773 之前現場聊天過，一直覺得本人比照片 223.141.125.149 01/13 13:26
→ wl00669773 漂亮 223.141.125.149 01/13 13:27
噓 hmt17 許光漢？ 49.216.52.98 01/13 14:26
推 WasJohnWall 派克？ 61.228.67.71 01/13 15:32
推 durian0308 讚 42.78.236.55 01/13 15:51
推 saw6904 楊謹華天花版 42.79.150.39 01/13 18:24
推 ruffryders 正翻 42.78.17.7 01/13 19:57
推 a3300689 已追蹤 推藏頭 1.171.153.189 01/14 00:32
→ a9564208 比較像 常威 101.9.96.122 01/14 07:33
推 clkdtm32 比楊謹華正太多了吧 198.232.126.7 01/14 11:00
推 ryanworld 有像 118.232.4.